Imports

In [ ]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
from QLCstuff2 import getNQLL
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

In [ ]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1

# Define initial conditions
Ntot_init_1D = np.ones(nx_crystal)
Nqll_init_1D = getNQLL(Ntot_init_1D,NSTAR,NBAR)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

In [ ]:
model = ip.IcePINN(num_hidden_layers=8, hidden_layer_size=80).to(device)

# Initialize model weights with HE initialization
def init_HE(m):
		if type(m) == nn.Linear:
			nn.init.kaiming_normal_(m.weight)
model.apply(init_HE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Train model

In [ ]:
ip.train_IcePINN(model, optimizer, training_set, epochs=50_000, print_every=1_000, print_gradients=False)

Save model

In [ ]:
print('Finished Training')
PATH = './models/TestPINN.pth'
torch.save(model.state_dict(), PATH)

Evaluate Model (visually?)

In [ ]:
# loaded_model = IcePINN()
# loaded_model.load_state_dict(torch.load(PATH)) # it takes the loaded dictionary, not the path file itself
# loaded_model.to(device)
# loaded_model.eval()